In [2]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn_image as isns
import calendar

In [3]:
import plotly.io as pio
pio.renderers.default='jupyterlab'

In [4]:
plt.style.use('seaborn-pastel')
isns.set_context(mode="notebook", fontfamily="Times")
sns.set_context(font_scale=0.8)

In [5]:
data_baseline = xr.open_mfdataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/sst_baseline_2.6_cropped.nc')

OSError: no files to open

In [ ]:
data_oae = xr.open_mfdataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/sst_OAE_2.6_cropped.nc')

In [ ]:
data_baseline = data_baseline.sst[0,:]
data_oae = data_oae.sst[0,:]

In [ ]:
data_baseline = data_baseline.to_dataset(name='sst')
data_oae = data_oae.to_dataset(name='sst')

In [ ]:
sliced_data_baseline = data_baseline.where((3 < data_baseline.nav_lon) & (data_baseline.nav_lon < 3.1) & (52 < data_baseline.nav_lat) & (data_baseline.nav_lat < 53), drop=True)
sliced_frame_baseline = sliced_data_baseline.sel(time_counter=slice('2090', '2100'))

In [ ]:
sliced_data_oae = data_oae.where((3 < data_oae.nav_lon) & (data_oae.nav_lon < 3.1) & (52 < data_oae.nav_lat) & (data_oae.nav_lat < 53), drop=True)
sliced_frame_oae = sliced_data_oae.sel(time_counter=slice('2090', '2100'))

### Define conditions

### Data Point Average of SST

In [ ]:
sliced_data_baseline_plot = (sliced_data_baseline.sst.mean(dim=('y', 'x'))).to_dataset(name='sst')
sliced_data_oae_plot = (sliced_data_oae.sst.mean(dim=('y', 'x'))).to_dataset(name='sst')

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(sliced_data_baseline_plot.time_counter, sliced_data_baseline_plot.sst, linewidth=2, label = 'Baseline')
plt.plot(sliced_data_oae_plot.time_counter, sliced_data_oae_plot.sst, linewidth=2, label = 'OAE')
plt.xlabel("Time", fontsize=14)
plt.ylabel("SST [°C]", fontsize=14)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=14)
plt.title('Data Point Average of SST', fontname="Times", fontsize=17, y=1.06)
plt.show()

In [ ]:
sliced_frame_baseline_plot = (sliced_frame_baseline.sst.mean(dim=('y', 'x'))).to_dataset(name='sst')
sliced_frame_oae_plot = (sliced_frame_oae.sst.mean(dim=('y', 'x'))).to_dataset(name='sst')

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(sliced_frame_baseline_plot.time_counter, sliced_frame_baseline_plot.sst, linewidth=2, label = 'Baseline')
plt.plot(sliced_frame_oae_plot.time_counter, sliced_frame_oae_plot.sst, linewidth=2, label = 'OAE')
plt.xlabel("Time", fontsize=14)
plt.ylabel("SST [°C]", fontsize=14)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=14)
plt.title('Data Point Average of SST', fontname="Times", fontsize=17, y=1.06)
plt.show()

### Unweighted Data Point Seasonal Average of SST

In [ ]:
# condition

start_date = "2090-01-01"
end_date = "2100-01-01"

In [ ]:
sst_baseline = sliced_data_baseline["sst"].sel(time_counter=slice(start_date, end_date))

In [ ]:
sst_seasonal_mean_baseline = (sst_baseline.resample(time_counter='QS-DEC').mean(keep_attrs=True)).mean(["y", "x"])

In [ ]:
sst_oae = sliced_data_oae["sst"].sel(time_counter=slice(start_date, end_date))

In [ ]:
sst_seasonal_mean_oae = (sst_oae.resample(time_counter='QS-DEC').mean(keep_attrs=True)).mean(["y", "x"])

In [ ]:
f, ax = plt.subplots(figsize=(20, 3))

sst_seasonal_mean_baseline.plot(marker="o", markersize=7, markeredgewidth=0.5,
                        color="black", linewidth=0.5,
                        markerfacecolor="lightskyblue", 
                        markeredgecolor="black", 
                        label = 'Baseline')

sst_seasonal_mean_oae.plot(marker="o", markersize=7, markeredgewidth=0.5,
                        color="black", linewidth=0.5,
                        markerfacecolor="lightgreen",
                        markeredgecolor="black", 
                        label = 'OAE')
ax.set_title("Unweighted Data Point Seasonal Average of SST", fontname="Times", fontsize=17, y=1.06)

plt.xlabel("Time", fontsize=14)
plt.ylabel("SST [°C] [°C]", fontsize=14)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=14)
plt.show()

### Weighted Data Point Seasonal Average of SST

In [ ]:
month_length_baseline = sst_baseline.time_counter.dt.days_in_month
sst_weighted_mean_baseline = ((sst_baseline * month_length_baseline).resample(time_counter='QS-DEC').sum() / month_length_baseline.resample(time_counter='QS-DEC').sum())
sst_weighted_season_baseline = (sst_weighted_mean_baseline.where(sst_weighted_mean_baseline)).mean(["y", "x"])

In [ ]:
month_length_oae = sst_oae.time_counter.dt.days_in_month
sst_weighted_mean_oae = ((sst_oae * month_length_oae).resample(time_counter='QS-DEC').sum() / month_length_oae.resample(time_counter='QS-DEC').sum())
sst_weighted_season_oae = (sst_weighted_mean_oae.where(sst_weighted_mean_oae)).mean(["y", "x"])

In [ ]:
f, ax = plt.subplots(figsize=(20, 3))
sst_weighted_season_baseline.plot(marker="o", markersize=7, markeredgewidth=0.5,
                        color="black", linewidth=0.5,
                        markerfacecolor="lightskyblue",
                        markeredgecolor="black", 
                        label = 'Baseline')

sst_weighted_season_oae.plot(marker="o", markersize=7, markeredgewidth=0.5,
                        color="black", linewidth=0.5,
                        markerfacecolor="lightgreen",
                        markeredgecolor="black", 
                        label = 'OAE')
ax.set_title("Weighted Data Point Seasonal Average of SST", fontname="Times", fontsize=17, y=1.06)

plt.xlabel("Time", fontsize=14)
plt.ylabel("SST [°C]", fontsize=14)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=14)
plt.show()

### Weighted Data Point Seasonal Average of SST (seasons displayed)

In [ ]:
# condition

colors = {3: "lightgreen", 6: "orange", 9: "brown", 12: "lightblue"}
seasons = {3: "Spring", 6: "Summer", 9: "Fall", 12: "Winter"}

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 8), sharey=False)
for month, arr in sst_weighted_season_baseline.groupby('time_counter.month'):
    arr.plot(ax=ax1,
             color="black", linewidth=0.5,
             marker="o", markersize=7, markeredgewidth=0.5,
             markerfacecolor=colors[month],
             markeredgecolor='black',
             label=seasons[month])

ax1.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=14)
ax1.set_title("Weighted Data Point Seasonal Average of SST without OAE", fontsize=17, y=1.06)
ax1.set_xlabel('Time', fontsize=14)
ax1.set_ylabel('SST [°C]', fontsize=14)

for month, arr in sst_weighted_season_oae.groupby('time_counter.month'):
    arr.plot(ax=ax2,
             color="black", linewidth=0.5,
             marker="o", markersize=7, markeredgewidth=0.5,
             markerfacecolor=colors[month],
             markeredgecolor='black',
             label=seasons[month])

ax2.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=14)
ax2.set_title("Weighted Data Point Seasonal Average of SST with OAE", fontname="Times", fontsize=17, y=1.06)
ax2.set_xlabel('Time', fontsize=14)
ax2.set_ylabel('SST [°C]', fontsize=14)
f.tight_layout()
plt.show()

In [ ]:
# define handlers for future legends

In [ ]:
class HandlerEllipse(HandlerPatch):
    def create_artists(self, legend, orig_handle,
                       xdescent, ydescent, width, height, fontsize, trans):
        center = 0.5 * width - 0.5 * xdescent, 0.5 * height - 0.5 * ydescent
        p = mpatches.Ellipse(xy=center, width=height + xdescent,
                             height=height + ydescent)
        self.update_prop(p, orig_handle, legend)
        p.set_transform(trans)
        return [p]

### Weighted Data Point Seasonal Average of SST without OAE (seasons displayed)

In [ ]:
sst_weighted_baseline = sst_weighted_season_baseline.to_dataframe('sst')
sst_weighted_baseline['Month'] = pd.DatetimeIndex(sst_weighted_baseline.index).month

In [ ]:
# condition

conditions_season = [
    (sst_weighted_baseline['Month'] == 3),
    (sst_weighted_baseline['Month'] == 6),
    (sst_weighted_baseline['Month'] == 9),
    (sst_weighted_baseline['Month'] == 12)
]

values_season = ['Spring', 'Summer', 'Fall', 'Winter']

In [ ]:
sst_weighted_baseline['Month'] = np.select(conditions_season, values_season)

In [ ]:
# condition

colors_season = dict(zip(sst_weighted_baseline.Month.unique(), ['paleturquoise', 'darkseagreen', 'gold', 'lightcoral']))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 3))
ax.plot_date(sst_weighted_baseline.index, sst_weighted_baseline.sst, '-', color="black",  linewidth=0.5, markeredgewidth=0.5, zorder=1)
ax.scatter(sst_weighted_baseline.index, sst_weighted_baseline.sst, color=sst_weighted_baseline.Month.map(lambda x: colors_season[x]), edgecolors='black',  linewidth=0.5, s=70)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_season.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=12, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('SST [°C]', fontsize=14)
ax.set_title("Weighted Data Point Seasonal Average of SST without OAE", fontname="Times", fontsize=17, y=1.06)

pass

### Weighted Data Point Seasonal Average of SST with OAE (seasons displayed)

In [ ]:
sst_weighted_oae = sst_weighted_season_oae.to_dataframe('sst')
sst_weighted_oae['Month'] = pd.DatetimeIndex(sst_weighted_oae.index).month

In [ ]:
sst_weighted_oae['Month'] = np.select(conditions_season, values_season)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 3))
ax.plot_date(sst_weighted_oae.index, sst_weighted_oae.sst, '-', color="black",  linewidth=0.5, zorder=1)
ax.scatter(sst_weighted_oae.index, sst_weighted_oae.sst, color=sst_weighted_oae.Month.map(lambda x: colors_season[x]), edgecolors='black',  linewidth=0.5, s=70)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_season.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=12, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Sea surface temperature ', fontsize=14)
ax.set_title("Weighted Data Point Seasonal Average of SST with OAE", fontname="Times", fontsize=17, y=1.06)

pass

### Weighted Data Point Seasonal Average of SST (seasons displayed)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 3))

ax.plot_date(sst_weighted_baseline.index, sst_weighted_baseline.sst, '-', color="black",  linewidth=0.5, zorder=1)
ax.text(43900, 8.1, s='Baseline')
ax.scatter(sst_weighted_baseline.index, sst_weighted_baseline.sst, color=sst_weighted_baseline.Month.map(lambda x: colors_season[x]), marker='o', edgecolors='black',  linewidth=0.5, s=70)

ax.plot_date(sst_weighted_oae.index, sst_weighted_oae.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.text(43900, 8.6, s='OAE')
ax.scatter(sst_weighted_oae.index, sst_weighted_oae.sst, color=sst_weighted_oae.Month.map(lambda x: colors_season[x]), marker='o', edgecolors='black', linewidth=0.5, s=70)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_season.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=12, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('SST [°C]', fontsize=14)
ax.set_title("Weighted Data Point Seasonal Average of SST", fontname="Times", fontsize=17, y=1.06)

pass

### Data Point Monthly Average of SST

In [ ]:
sst_monthly_mean_baseline = (sliced_data_baseline["sst"].sel(time_counter=slice(start_date, end_date))).mean(["y", "x"])

In [ ]:
sst_monthly_mean_oae = (sliced_data_oae["sst"].sel(time_counter=slice(start_date, end_date))).mean(["y", "x"])

In [ ]:
f, ax = plt.subplots(figsize=(20, 3))

sst_monthly_mean_baseline.plot(marker="o", markersize=5, markeredgewidth=0.5,
                        color="black", linewidth=0.5,
                        markerfacecolor="lightskyblue", 
                        markeredgecolor="black", 
                        label = 'Baseline')

sst_monthly_mean_oae.plot(marker="o", markersize=5, markeredgewidth=0.5,
                        color="black", linewidth=0.5,
                        markerfacecolor="lightgreen",
                        markeredgecolor="black", 
                        label = 'OAE')
ax.set_title("Data Point Monthly Average of SST without OAE", fontname="Times", fontsize=17, y=1.06)

plt.xlabel("Time", fontsize=14)
plt.ylabel("SST [°C]", fontsize=14)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=14)
plt.show()

### Data Point Monthly Average of SST without OAE (months displayed)

In [ ]:
sst_baseline_monthly = sst_monthly_mean_baseline.to_dataframe('sst')
sst_baseline_monthly['Month'] = pd.DatetimeIndex(sst_baseline_monthly.index).month

In [ ]:
# condition

conditions_month = [
    (sst_baseline_monthly['Month'] == 1),
    (sst_baseline_monthly['Month'] == 2),
    (sst_baseline_monthly['Month'] == 3),
    (sst_baseline_monthly['Month'] == 4),
    (sst_baseline_monthly['Month'] == 5),
    (sst_baseline_monthly['Month'] == 6),
    (sst_baseline_monthly['Month'] == 7),
    (sst_baseline_monthly['Month'] == 8),
    (sst_baseline_monthly['Month'] == 9),
    (sst_baseline_monthly['Month'] == 10),
    (sst_baseline_monthly['Month'] == 11),
    (sst_baseline_monthly['Month'] == 12)
]

values_month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']

In [ ]:
sst_baseline_monthly['Month'] = np.select(conditions_month, values_month)

In [ ]:
# condition

colors_month = dict(zip(sst_baseline_monthly.Month.unique(), ['powderblue', 'darkcyan', 'forestgreen', 'darkseagreen', 
                                                        'honeydew', 'bisque', 'khaki', 'gold', 
                                                        'sandybrown', 'lightcoral', 'chocolate', 'cadetblue']))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 3))
ax.plot_date(sst_baseline_monthly.index, sst_baseline_monthly.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.scatter(sst_baseline_monthly.index, sst_baseline_monthly.sst, color=sst_baseline_monthly.Month.map(lambda x: colors_month[x]), edgecolors='black',  linewidth=0.5, s=40)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_month.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1.3), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('SST [°C]', fontsize=14)
ax.set_title("Data Point Monthly Average of SST without OAE", fontname="Times", fontsize=17, y=1.06)

pass

### Data Point Monthly Average of SST with OAE (months displayed)

In [ ]:
sst_oae_monthly = sst_monthly_mean_oae.to_dataframe('sst')
sst_oae_monthly['Month'] = pd.DatetimeIndex(sst_oae_monthly.index).month

In [ ]:
sst_oae_monthly['Month'] = np.select(conditions_month, values_month)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 3))
ax.plot_date(sst_oae_monthly.index, sst_oae_monthly.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.scatter(sst_oae_monthly.index, sst_oae_monthly.sst, color=sst_oae_monthly.Month.map(lambda x: colors_month[x]), edgecolors='black',  linewidth=0.5, s=40)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_month.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 4, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_ylabel('SST [°C]', fontsize=14)
#ax.set_title("Data Point Monthly Average of SST with OAE", fontname="Times", fontsize=17, y=1.06)

pass

### Data Point Monthly Average of SST (months displayed)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 3))

ax.plot_date(sst_baseline_monthly.index, sst_baseline_monthly.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.text(43750, 8.2, s='Baseline')
ax.scatter(sst_baseline_monthly.index, sst_baseline_monthly.sst, color=sst_baseline_monthly.Month.map(lambda x: colors_month[x]), marker='o', edgecolors='black', linewidth=0.5, s=40)

ax.plot_date(sst_oae_monthly.index, sst_oae_monthly.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.text(43800, 8.6, s='OAE')
ax.scatter(sst_oae_monthly.index, sst_oae_monthly.sst, color=sst_oae_monthly.Month.map(lambda x: colors_month[x]), marker='o', edgecolors='black', linewidth=0.5, s=40)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_month.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('SST [°C]', fontsize=14)
ax.set_title("Data Point Monthly Average of SST", fontsize=17, y=1.06)

pass

In [ ]:
mean_baseline = (sst_baseline_monthly.groupby(sst_baseline_monthly.index.month).sst.mean()).reset_index()
mean_baseline['time_counter'] = mean_baseline['time_counter'].apply(lambda x: calendar.month_abbr[x])

In [ ]:
mean_oae = (sst_oae_monthly.groupby(sst_oae_monthly.index.month).sst.mean()).reset_index()
mean_oae['time_counter'] = mean_oae['time_counter'].apply(lambda x: calendar.month_abbr[x])

In [ ]:
# condition

colors_time = dict(zip(mean_baseline.time_counter.unique(), ['powderblue', 'darkcyan', 'forestgreen', 'darkseagreen', 
                                                        'honeydew', 'bisque', 'khaki', 'gold', 
                                                        'sandybrown', 'lightcoral', 'chocolate', 'cadetblue']))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(mean_baseline.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.scatter(mean_baseline.time_counter, mean_baseline.sst, color=mean_baseline.time_counter.map(lambda x: colors_time[x]), edgecolors='black',  linewidth=0.5, s=30)

ax.plot(mean_oae.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.scatter(mean_oae.time_counter, mean_oae.sst, color=mean_oae.time_counter.map(lambda x: colors_time[x]), edgecolors='black',  linewidth=0.5, s=30)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_time.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})
ax.set_ylabel('SST [°C]', fontsize=10)

ax.tick_params(axis='both', which='major', labelsize=10)
ax.yaxis.get_offset_text().set_fontsize(10)

ax.set_title("EU Monthly Average of SST over the last decade", fontname="Times", fontsize=14, y=1.06)

pass

### Δ SST between Baseline and OAE

In [ ]:
delta_sst = (sst_oae_monthly.sst - sst_baseline_monthly.sst).to_xarray()

In [ ]:
delta_sst = delta_sst.to_dataframe('sst')
delta_sst['Month'] = pd.DatetimeIndex(delta_sst.index).month

In [ ]:
delta_sst['Month'] = np.select(conditions_month, values_month)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 3))
ax.plot_date(delta_sst.index, delta_sst.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.scatter(delta_sst.index, delta_sst.sst, color=delta_sst.Month.map(lambda x: colors_month[x]), edgecolors='black',  linewidth=0.5, s=40)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_month.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Δ SST', fontsize=14)
ax.set_title("Δ SST", fontname="Times", fontsize=17, y=1.06)

pass

### Averaged monthly cycle of Δ SST

In [ ]:
delta_sst_mean = (delta_sst.groupby(delta_sst.index.month).sst.mean()).reset_index()
delta_sst_mean['time_counter'] = delta_sst_mean['time_counter'].apply(lambda x: calendar.month_abbr[x])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(delta_sst_mean.sst, '-', color="black", linewidth=0.5, zorder=1)
ax.scatter(delta_sst_mean.time_counter, delta_sst_mean.sst, color=delta_sst_mean.time_counter.map(lambda x: colors_time[x]), edgecolors='black',  linewidth=0.5, s=30)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_month.items()]
ax.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax.set_ylabel('Δ SST', fontsize=10)
ax.tick_params(axis='both', which='major', labelsize=10)
ax.set_title("EU Monthly Average of Δ SST over the last decade", fontname="Times", fontsize=14, y=1.06)

pass

In [ ]:
f, (ax, ax2) = plt.subplots(1,2, figsize=(20, 3), gridspec_kw={'width_ratios': [6, 1]}, sharey=True) 
plt.subplots_adjust(wspace=0, hspace=0)


ax.plot_date(delta_sst.index, delta_sst.sst, '-', label='Baseline', color="black", linewidth=0.5, zorder=0.8)
ax.scatter(delta_sst.index, delta_sst.sst, color=delta_sst.Month.map(lambda x: colors_month[x]), marker='o', edgecolors='black', linewidth=0.5, s=30)

ax.tick_params(axis='both', which='major', labelsize=10)
ax.set_ylabel('Δ SST', fontsize=10)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_month.items()]
ax2.legend(handles = legend, bbox_to_anchor=(1, 1), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 1, handler_map={mpatches.Circle: HandlerEllipse()})

ax2.plot(delta_sst_mean.sst, '-', color="black", linewidth=0.5, zorder=1)
ax2.scatter(delta_sst_mean.time_counter, delta_sst_mean.sst, color=delta_sst_mean.time_counter.map(lambda x: colors_time[x]), edgecolors='black',  linewidth=0.5, s=20)

ax2.tick_params(axis='x', which='major', labelsize=7)

#plt.savefig('/Users/chiaraciscato/Desktop/geomar/geomarcode/SSP1_2.6/DPsst_change_monthly_mean.png')

pass

In [ ]:
f, ((ax, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(20, 7), gridspec_kw={'width_ratios': [6, 1], 'height_ratios': [3, 2]}, sharey='row')  
plt.subplots_adjust(wspace=0, hspace=0)

ax.plot_date(sst_baseline_monthly.index, sst_baseline_monthly.sst, '-', label='Baseline', color="black", linewidth=0.5, zorder=0.8)
ax.text(43750, 8.15, s='Baseline', fontsize=12)
ax.scatter(sst_baseline_monthly.index, sst_baseline_monthly.sst, color=sst_baseline_monthly.Month.map(lambda x: colors_month[x]), marker='o', edgecolors='black', linewidth=0.5, s=30)

ax.plot_date(sst_oae_monthly.index, sst_oae_monthly.sst, '-', label='OAE', color="black", linewidth=0.5, zorder=1)
ax.text(43800, 8.58, s='OAE', fontsize=12)
ax.scatter(sst_oae_monthly.index, sst_oae_monthly.sst, color=sst_oae_monthly.Month.map(lambda x: colors_month[x]), marker='o', edgecolors='black', linewidth=0.5, s=30)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.set_ylabel("SST [°C]", fontsize=12)

legend = [mpatches.Circle((0.5, 0.5), 1, facecolor = value, edgecolor = "black", linewidth=0.5, label = key, alpha = 1) for key, value in colors_month.items()]
ax.legend(handles = legend, bbox_to_anchor=(1.17, 1.21), loc='best', columnspacing=0.05, handletextpad=0.4, fontsize=9, ncol = 4, handler_map={mpatches.Circle: HandlerEllipse()})

ax2.plot(mean_baseline.sst, '-', color="black", linewidth=0.5, zorder=1)
ax2.scatter(mean_baseline.time_counter, mean_baseline.sst, color=mean_baseline.time_counter.map(lambda x: colors_time[x]), edgecolors='black',  linewidth=0.5, s=30)

ax2.plot(mean_oae.sst, '-', color="black", linewidth=0.5, zorder=1)
ax2.scatter(mean_oae.time_counter, mean_oae.sst, color=mean_oae.time_counter.map(lambda x: colors_time[x]), edgecolors='black',  linewidth=0.5, s=30)

ax2.tick_params(axis='x', which='major', labelsize=8)
ax2.tick_params(axis='y', which='major', labelsize=12)

ax3.plot_date(delta_sst.index, delta_sst.sst, '-', label='Baseline', color="black", linewidth=0.5, zorder=0.8)
ax3.scatter(delta_sst.index, delta_sst.sst, color=delta_sst.Month.map(lambda x: colors_month[x]), marker='o', edgecolors='black', linewidth=0.5, s=30)

ax3.tick_params(axis='both', which='major', labelsize=12)
ax3.set_ylabel('Δ SST', fontsize=12)

ax4.plot(delta_sst_mean.sst, '-', color="black", linewidth=0.5, zorder=1)
ax4.scatter(delta_sst_mean.time_counter, delta_sst_mean.sst, color=delta_sst_mean.time_counter.map(lambda x: colors_time[x]), edgecolors='black',  linewidth=0.5, s=20)

ax4.tick_params(axis='x', which='major', labelsize=10, rotation=90)
ax4.tick_params(axis='y', which='major', labelsize=12)

#plt.savefig('/Users/chiaraciscato/Desktop/geomar/geomarcode/SSP1_2.6/DPsst_monthly_mean.png')

pass